# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [39]:
# load data from database
engine = create_engine('sqlite:///udacity_dsnd.db')
df = pd.read_sql("InsertTableName", con = engine)

In [40]:
X = df["message"]
Y = df.iloc[:,4:].astype(int)

### 2. Write a tokenization function to process your text data

In [41]:
def tokenize(text):
    #detected_urls = re.findall(url_regex, text)
  #  for url in detected_urls:
     #   text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [42]:
#to remove columns with all zeros
#Y.sum(axis=0)
#check if a column has zero 1s. This category column will be dropped off. 
#remove rows with all zeros if intended
#X = X[Y.sum(axis=1)!=0]
#Y = Y[Y.sum(axis=1)!=0]

In [43]:
for column in Y.columns:
    Y[column] =Y[column].apply(lambda x : 1 if x > 1 else x)
    if Y[column].sum(axis=0)==0:
        Y =Y.drop(column, axis=1)

In [44]:
category_names = Y.columns

In [45]:
Y = np.array(Y)
X = np.array(X)
X = X.reshape(len(X),1)
X = X.flatten()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [46]:

pipeline = Pipeline(steps= [("vect",CountVectorizer(tokenizer=tokenize)),
                            #("transformer",TfidfTransformer(smooth_idf=False)),
                            ("estimator",MultiOutputClassifier(LogisticRegression()))
                           ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size =0.4, random_state=77)

In [48]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
score = []
for i in range (len(category_names)):
    mask = np.array([y_test[:,i] == 1])
    y_test_sub = pd.DataFrame(y_test)
    mask =mask.flatten()
    index = y_test_sub.iloc[mask,i].index
    y_test_sub = np.array(y_test_sub)[index]
    X_test_sub = X_test[index]
    y_pred_sub = pipeline.predict(X_test_sub)
    score.append(accuracy_score(y_test_sub[:,i],y_pred_sub[:,i]))
    print(classification_report(y_test_sub,y_pred_sub))

             precision    recall  f1-score   support

          0       1.00      0.91      0.95      8015
          1       0.79      0.58      0.67      1828
          2       0.00      0.00      0.00        45
          3       0.79      0.66      0.72      4385
          4       0.60      0.30      0.40       819
          5       0.64      0.29      0.40       538
          6       0.64      0.17      0.27       291
          7       0.21      0.02      0.04       179
          8       0.67      0.28      0.40       351
          9       0.76      0.58      0.66       665
         10       0.84      0.70      0.77      1182
         11       0.80      0.55      0.65       961
         12       0.84      0.26      0.40       175
         13       0.66      0.22      0.33       260
         14       0.65      0.10      0.18       127
         15       0.54      0.20      0.29       367
         16       0.78      0.47      0.59       485
         17       0.46      0.21      0.29   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      0.97      0.98      1828
          1       1.00      0.58      0.74      1828
          2       0.00      0.00      0.00         0
          3       0.95      0.75      0.84      1657
          4       0.72      0.27      0.39       275
          5       0.78      0.31      0.45       185
          6       0.44      0.09      0.16        85
          7       0.00      0.00      0.00        61
          8       0.25      0.05      0.08        20
          9       0.89      0.69      0.78       356
         10       0.97      0.84      0.90       647
         11       0.91      0.63      0.75       492
         12       1.00      0.21      0.35        81
         13       0.83      0.19      0.31        80
         14       0.50      0.07      0.12        28
         15       0.44      0.11      0.17        76
         16       0.89      0.38      0.53        90
         17       0.61      0.28      0.39   

In [50]:
np.mean(score)

0.32639138025537512

### 6. Improve your model
Use grid search to find better parameters. 

In [51]:
parameters = {
    'estimator__estimator__penalty': ["l1","l2"],
    'estimator__estimator__C': [0.5,0.75],
    'vect__ngram_range': [(1, 1), (2, 2)],
  #  'tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline,param_grid=parameters,refit = True)

In [52]:
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'estimator__estimator__penalty': ['l1', 'l2'], 'estimator__estimator__C': [0.5, 0.75], 'vect__ngram_range': [(1, 1), (2, 2)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [53]:
cv.best_params_

{'estimator__estimator__C': 0.5,
 'estimator__estimator__penalty': 'l1',
 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [54]:
grid_search_score = []
for i in range (len(category_names)):
    mask = np.array([y_test[:,i] == 1])
    y_test_sub2 = pd.DataFrame(y_test)
    mask =mask.flatten()
    index = y_test_sub2.iloc[mask,i].index
    y_test_sub2 = np.array(y_test_sub2)[index]
    X_test_sub2 = X_test[index]
    y_pred_sub2 = cv.predict(X_test_sub2)
    grid_search_score.append(accuracy_score(y_test_sub2[:,i],y_pred_sub2[:,i]))
    print(classification_report(y_test_sub2,y_pred_sub2))

             precision    recall  f1-score   support

          0       1.00      0.93      0.96      8015
          1       0.81      0.56      0.66      1828
          2       0.00      0.00      0.00        45
          3       0.81      0.65      0.72      4385
          4       0.64      0.26      0.37       819
          5       0.68      0.30      0.42       538
          6       0.72      0.17      0.27       291
          7       0.27      0.03      0.06       179
          8       0.69      0.30      0.42       351
          9       0.79      0.57      0.66       665
         10       0.85      0.73      0.79      1182
         11       0.83      0.54      0.66       961
         12       0.82      0.36      0.50       175
         13       0.67      0.27      0.38       260
         14       0.76      0.20      0.31       127
         15       0.59      0.22      0.32       367
         16       0.78      0.52      0.62       485
         17       0.53      0.15      0.23   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      0.98      0.99      1828
          1       1.00      0.56      0.71      1828
          2       0.00      0.00      0.00         0
          3       0.95      0.76      0.85      1657
          4       0.78      0.21      0.33       275
          5       0.84      0.36      0.50       185
          6       0.50      0.05      0.09        85
          7       1.00      0.02      0.03        61
          8       0.50      0.10      0.17        20
          9       0.93      0.70      0.80       356
         10       0.97      0.87      0.92       647
         11       0.92      0.61      0.74       492
         12       0.93      0.31      0.46        81
         13       0.82      0.23      0.35        80
         14       0.50      0.04      0.07        28
         15       0.60      0.12      0.20        76
         16       0.80      0.41      0.54        90
         17       0.65      0.23      0.34   

In [55]:
np.mean(grid_search_score)

0.33446573713185046

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [56]:
from sklearn.linear_model import SGDClassifier
improved_pipeline = Pipeline(steps= [("tokenizer",CountVectorizer(tokenizer=tokenize)),
                            ("transformer",TfidfTransformer()),
                            ("estimator",MultiOutputClassifier(SGDClassifier(loss='hinge', penalty='l2',
                                                                             alpha=1e-3, random_state=42 ,max_iter=10, tol=None)))])

In [57]:
improved_pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tokenizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        ...random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False),
           n_jobs=1))])

In [58]:
new_score = []
for i in range (len(category_names)):
    mask = np.array([y_test[:,i] == 1])
    y_test_sub = pd.DataFrame(y_test)
    mask =mask.flatten()
    index = y_test_sub.iloc[mask,i].index
    y_test_sub = np.array(y_test_sub)[index]
    X_test_sub = X_test[index]
    y_pred_sub = improved_pipeline.predict(X_test_sub)
    new_score.append(accuracy_score(y_test_sub[:,i],y_pred_sub[:,i]))
    print(classification_report(y_test_sub,y_pred_sub))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      0.99      1.00      8015
          1       0.90      0.43      0.59      1828
          2       0.00      0.00      0.00        45
          3       0.85      0.45      0.59      4385
          4       0.00      0.00      0.00       819
          5       0.00      0.00      0.00       538
          6       0.00      0.00      0.00       291
          7       0.00      0.00      0.00       179
          8       0.00      0.00      0.00       351
          9       0.84      0.05      0.09       665
         10       0.87      0.39      0.53      1182
         11       0.91      0.08      0.15       961
         12       0.00      0.00      0.00       175
         13       0.00      0.00      0.00       260
         14       0.00      0.00      0.00       127
         15       0.00      0.00      0.00       367
         16       0.00      0.00      0.00       485
         17       0.00      0.00      0.00   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1828
          1       1.00      0.43      0.60      1828
          2       0.00      0.00      0.00         0
          3       0.95      0.66      0.78      1657
          4       0.00      0.00      0.00       275
          5       0.00      0.00      0.00       185
          6       0.00      0.00      0.00        85
          7       0.00      0.00      0.00        61
          8       0.00      0.00      0.00        20
          9       0.96      0.07      0.13       356
         10       0.99      0.53      0.69       647
         11       0.99      0.14      0.25       492
         12       0.00      0.00      0.00        81
         13       0.00      0.00      0.00        80
         14       0.00      0.00      0.00        28
         15       0.00      0.00      0.00        76
         16       0.00      0.00      0.00        90
         17       0.00      0.00      0.00   

In [59]:
np.mean(new_score)

0.10410545153679518

In [60]:
model_selected = cv

### 8a. Trying out prediction

In [61]:
query =  "water supplies requested"
classification_labels = pipeline.predict([query])[0]
print(dict(zip(df.columns[4:], classification_labels)))

{'related': 1, 'request': 0, 'offer': 0, 'aid_related': 0, 'medical_help': 0, 'medical_products': 0, 'search_and_rescue': 0, 'security': 0, 'military': 0, 'child_alone': 0, 'water': 0, 'food': 0, 'shelter': 0, 'clothing': 0, 'money': 0, 'missing_people': 0, 'refugees': 0, 'death': 0, 'other_aid': 0, 'infrastructure_related': 0, 'transport': 0, 'buildings': 0, 'electricity': 0, 'tools': 0, 'hospitals': 0, 'shops': 0, 'aid_centers': 0, 'other_infrastructure': 0, 'weather_related': 0, 'floods': 0, 'storm': 0, 'fire': 0, 'earthquake': 0, 'cold': 0, 'other_weather': 0}


### 9. Export your model as a pickle file

In [62]:
with open("model.pkt", "wb") as f:
    pickle.dump(model_selected, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.